In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Flatten,Conv1D,MaxPooling1D,Dropout
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zipfile
from numpy import array
from numpy import hstack

In [2]:
#zip = zipfile.ZipFile('combined_df_1030.zip')
#zip.extractall()

In [3]:
#zip = zipfile.ZipFile('combined_df_2030.zip')
#zip.extractall()

In [4]:
#zip = zipfile.ZipFile('combined_df_3030.zip')
#zip.extractall()

In [5]:
blocks = [1030, 2030, 3030]
#blocks = [1030, 2030, 3030, 4030, 5030, 6030, 7030]

dfss = []
for block in blocks:
    print(block)
    dfss.append(pd.read_csv('combined_df_'+str(block)+'.csv'))

1030
2030
3030


In [6]:
df = pd.concat(dfss)

In [7]:
#df.to_csv('full_blisk.csv', mode='a', chunksize='10000',index=False)

In [8]:
#df = pd.read_csv('full_blisk.csv')
df.shape

(1685106, 71)

In [9]:
df.columns

Index(['Unnamed: 0', 'Leading angle', 'Side tilt angle',
       'Surface Contact point X', 'Surface contact point Y',
       'Surface contact point z', 'Surface Orietation X',
       'Surface Orietation Y', 'Surface Orietation Z', 'Tool Tip Point X',
       'Tool Tip Point Y', 'Tool Tip Point Z', 'Tool Orientation X',
       'Tool Orientation Y', 'Tool Orientation Z',
       'Closes Point Distance (Surface Contact Point - Acquired TCP 1030 blade0)',
       'time', 'MachineX', 'MachineY', 'MachineZ', 'MachineA', 'MachineC',
       'LoadSpindle', 'TransfX', 'TransfY', 'TransfZ', 'TransfI', 'TransfJ',
       'TransfK', 'executionDuration', 'previousExecutionDuration',
       'relativeTimetoPreviousCall program number', 'timeStamp', 'Tool Length',
       'Tool Radius', 'G54-X', 'G54-Y', 'G54-Z', 'G54-A', 'G54-C',
       'ProgNumber', 'Blade', 'Fx_mean', 'Fy_mean', 'Fz_mean', 'Mz_mean',
       'Fc_mean', 'Fa_mean', 'FcN_mean', 'Fres_mean', 'Fx_sigma', 'Fy_sigma',
       'Fz_sigma', 'Mz_sigm

In [10]:
def get_dataset():
    X = df[['Leading angle','Side tilt angle','Tool Tip Point X',
           'Tool Tip Point Y', 'Tool Tip Point Z', 'Tool Orientation X',
           'Tool Orientation Y', 'Tool Orientation Z',]].copy(deep=True).to_numpy()
    y = df[['MachineX', 'MachineY', 'MachineZ', 'MachineA', 'MachineC']].copy(deep=True).to_numpy()
    
    return X,y

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [18]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(30, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X[0:100]):
        print('Train index:', train_ix.shape,'Test_index',test_ix.shape)
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        
        # fit model
        print('model fit')
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        print('model evaluate')
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('mae:',mae)
        results.append(mae)
    return results
 
# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X[:,:], y[:,:])
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Train index: (66,) Test_index (34,)
model fit
model evaluate
mae: 146.65322875976562
Train index: (67,) Test_index (33,)
model fit
model evaluate
mae: 146.33497619628906
Train index: (67,) Test_index (33,)
model fit
model evaluate
mae: 144.3882598876953
MAE: 145.792 (1.001)


In [18]:
np.sqrt(18.504)

4.301627598944381

In [15]:
"""
from numpy import asarray
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
 
# get the dataset
def get_dataset():
    X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3, random_state=2)
    return X, y
 
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
    model.compile(loss='mae', optimizer='adam')
    return model
 
# load dataset
X, y = get_dataset()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose=0, epochs=100)
# make a prediction for new data
row = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]
newX = asarray([row])
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])
"""

"\nfrom numpy import asarray\nfrom sklearn.datasets import make_regression\nfrom keras.models import Sequential\nfrom keras.layers import Dense\n \n# get the dataset\ndef get_dataset():\n    X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3, random_state=2)\n    return X, y\n \n# get the model\ndef get_model(n_inputs, n_outputs):\n    model = Sequential()\n    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))\n    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))\n    model.compile(loss='mae', optimizer='adam')\n    return model\n \n# load dataset\nX, y = get_dataset()\nn_inputs, n_outputs = X.shape[1], y.shape[1]\n# get model\nmodel = get_model(n_inputs, n_outputs)\n# fit the model on all data\nmodel.fit(X, y, verbose=0, epochs=100)\n# make a prediction for new data\nrow = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]\nnewX = 

In [1]:
!pip install tensorflow

  Using cached tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl (454.3 MB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached protobuf-3.17.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached absl_py-0.13.0-py3-none-any.whl (132 kB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
Processing /home/jovyan/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2/termcolor-1.1.0-py3-none-any.whl
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached grpcio-1.34.1-cp37-cp37m-manylinux2014_x86_64.whl (4.0 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
Processing /home/jovyan/.cache/pip/wheels/62/76/4c/aa25

In [2]:
!pip install pandas

  Using cached pandas-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (10.8 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)


In [3]:
!pip install sklearn

Processing /home/jovyan/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached scipy-1.7.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.5 MB)
  Using cached threadpoolctl-2.2.0-py3-none-any.whl (12 kB)


In [4]:
!pip install h5py

In [5]:
!pip install matplotlib

In [6]:
!pip install PyAstronomy

Processing /home/jovyan/.cache/pip/wheels/fb/cd/72/74c0769291063761fb050b8f05b80f5603789c2374e80d51d1/PyAstronomy-0.16.0-py3-none-any.whl


In [7]:
!pip install Scipy

In [8]:
!pip install matplotlib

In [9]:
!pip install keras-tuner --upgrade

  Using cached keras_tuner-1.0.3-py3-none-any.whl (96 kB)
  Using cached packaging-21.0-py3-none-any.whl (40 kB)
Processing /home/jovyan/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c/kt_legacy-1.0.3-py3-none-any.whl


In [14]:
!pip install pydot

In [15]:
!pip install graphviz

In [1]:
import tensorflow as tf
tf.__version__

'2.3.1'

In [3]:
from platform import python_version
print(python_version())

3.7.9
